<a href="https://colab.research.google.com/github/yukiao/Pengantar-Deep-Learning/blob/main/Week3/Chapter_5_Deep_Learning_Computation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Computation

## Layers and Blocks

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
# Sebuah neural network sederhana yang terdiri dari 1 hidden layer dengan 256 neuron
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X  = torch.rand(2, 20)
net(X)

tensor([[-0.0170,  0.0209,  0.0100,  0.0521, -0.0965, -0.1227, -0.1189, -0.0351,
         -0.1753,  0.0781],
        [-0.0571,  0.0372,  0.0240, -0.0328, -0.1161, -0.0466, -0.0496,  0.0810,
         -0.1783,  0.1381]], grad_fn=<AddmmBackward0>)

### A Custom Block

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256) # Hidden layer yang terdiri dari 256 neuron
        self.out = nn.Linear(256, 10) # Output layer dengan 10 output neuron
        
    # Method forward adalah method yang digunakan untuk melakukan forward propagation pada model
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
net(X)

tensor([[-0.1965, -0.0319, -0.0101, -0.0828, -0.0091,  0.0089, -0.0118,  0.1880,
          0.1247,  0.3202],
        [-0.2444,  0.0770, -0.1106,  0.0511,  0.1029, -0.1459, -0.0055,  0.0699,
         -0.0628,  0.0876]], grad_fn=<AddmmBackward0>)

In [5]:
### The Sequential Block

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [7]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0638,  0.0324,  0.2623, -0.2685,  0.0379,  0.2386, -0.1494, -0.0483,
          0.3227,  0.0246],
        [-0.0149,  0.0122,  0.1605, -0.0335,  0.1348,  0.2370, -0.0214, -0.1536,
          0.1315, -0.0022]], grad_fn=<AddmmBackward0>)

### Executing Code in the Forward Propagation Function

In [8]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Paramater weight acak yang diinisialisasi kemudian nilai dari weight ini tidak akan diupdate sehingga bersifat constant
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
        
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [9]:
net = FixedHiddenMLP()
net(X)

tensor(-0.0568, grad_fn=<SumBackward0>)

In [10]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
        nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0895, grad_fn=<SumBackward0>)

### Efficiency

## Parameter Management

In [11]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.2650],
        [0.2232]], grad_fn=<AddmmBackward0>)

### Parameter Access

In [13]:
print(net[2].state_dict())
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

OrderedDict([('weight', tensor([[ 0.3456,  0.1517, -0.0114,  0.1889,  0.1373, -0.1386, -0.1087, -0.2349]])), ('bias', tensor([0.0284]))])
<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.0284], requires_grad=True)
tensor([0.0284])


In [14]:
net[2].weight.grad == None

True

In [15]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [16]:
net.state_dict()['2.bias'].data

tensor([0.0284])

In [17]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.2884],
        [-0.2889]], grad_fn=<AddmmBackward0>)

In [18]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


### Parameter Initialization

In [19]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0046,  0.0075,  0.0007, -0.0070]), tensor(0.))

In [20]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
    net.apply(init_constant)
    net[0].weight.data[0], net[0].bias.data[0]

In [21]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
        
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.6001, -0.6263,  0.3703,  0.0994])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [22]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5
        
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000,  5.1546, -0.0000,  8.9912],
        [-0.0000, -0.0000,  6.5680, -5.9196]], grad_fn=<SliceBackward0>)

In [23]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  6.1546,  1.0000,  9.9912])

### Tied Parameters

In [25]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)

tensor([[-0.3471],
        [-0.3313]], grad_fn=<AddmmBackward0>)

In [26]:
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## Custom Layers

### Layers without Parameters

In [27]:
import torch
from torch import nn
from torch.nn import functional as F

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, X):
        return X - X.mean()

In [28]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [29]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [30]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(0., grad_fn=<MeanBackward0>)

### Layers with Parameters

In [31]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
        
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [32]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.2411, -0.0816, -0.9063],
        [-0.5038,  0.1378, -0.5157],
        [ 1.2594,  0.6264, -2.5091],
        [ 0.6270,  0.7953, -0.1660],
        [-0.6710,  0.2293,  0.8547]], requires_grad=True)

In [33]:
linear(torch.rand(2, 5))

tensor([[0.0000, 2.3019, 0.0000],
        [0.2446, 1.8841, 0.0000]])

In [34]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[10.9826],
        [ 3.2733]])

## File I/O

### Loading and Saving Tensors

In [35]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

In [36]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [37]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [38]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

### Loading and Saving Model Parameters

In [39]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [40]:
torch.save(net.state_dict(), 'mlp.params')

In [41]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [42]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

## GPUs

In [43]:
!nvidia-smi

Fri Apr  8 23:49:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Computing Devices

In [44]:
import torch
from torch import nn
torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [45]:
torch.cuda.device_count()

1

In [51]:
def try_gpu(i=0): 
    """Return gpu(i) if exists, otherwise return cpu()."""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus(): 
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    devices = [torch.device(f'cuda:{i}')
               for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

### Tensors and GPUs

In [52]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [53]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

Berbeda dengan contoh yang ada pada buku Dive to Deep Learning, disini kita hanya memiliki 1 buah GPU

In [62]:
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.9701, 0.2748, 0.0460],
        [0.9033, 0.2096, 0.7527]], device='cuda:0')

In [66]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [67]:
Y + Z

tensor([[1.9701, 1.2748, 1.0460],
        [1.9033, 1.2096, 1.7527]], device='cuda:0')

In [69]:
Z.cuda(0) is Z

True

### Neural Networks and GPUs

In [71]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [72]:
net(X)

tensor([[0.3586],
        [0.3586]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [73]:
net[0].weight.data.device

device(type='cuda', index=0)